In [1]:
import pandas as pd
import numpy as np
import sklearn.svm
from sklearn.model_selection import train_test_split
import random
import time

### Data Preprocessing

In [2]:
class DataPreprocess:
    def __init__(self) -> None:
        pass
    
    # 處理 data.csv
    # 用pandas讀資料，處理後再分開X和Y
    def data_prepare_1(self, file_path):
        df_file = pd.read_csv(file_path)
        Y = df_file['Diagnosis'].map({'M':-1, 'B':1}).to_numpy()      
        X = df_file.iloc[:,2:].to_numpy()
        return X, Y

    # 處理 crx.csv
    def data_prepare_2(self, file_path):
        df_file = pd.read_csv(file_path)
        Y = df_file['label'].map({'+':1, '-':-1}).to_numpy()
        X = df_file.iloc[:,:15]
        X['att1'] = X['att1'].map({'?':0, 'a':-1, 'b':1})
        X['att2'] = X['att2'].replace('?', 0)
        X['att2'] = X['att2'].astype(float)
        X['att4'] = X['att4'].map({'?':0, 'u':-1, 'y':1, 'l':2})
        X['att5'] = X['att5'].map({'?':0, 'g':-1, 'p':1, 'gg':2})
        X['att6'] = X['att6'].map({'?':0, 'c':-7, 'q':-6, 'w':-5, 'i':-4, 'aa':-3,
                        'ff':-2, 'k':-1, 'cc':1, 'm':2, 'x':3, 'd':4,
                        'e':5, 'j':6, 'r':7})
        X['att7'] = X['att7'].map({'?':0, 'v':-4, 'h':-3, 'bb':-2, 'ff':-1, 'j':1,
                        'z':2, 'dd':3, 'n':4, 'o':5})
        X['att9'] = X['att9'].map({'t':-1, 'f':1})
        X['att10'] = X['att10'].map({'t':-1, 'f':1})
        X['att12'] = X['att12'].map({'t':-1, 'f':1})
        X['att13'] = X['att13'].map({'g':0, 's':-1, 'p':1})
        X['att14'] = X['att14'].replace('?', 0)
        X['att14'] = X['att14'].astype(float)
        X = X.to_numpy()
        return X, Y

### Linear Classifier

In [11]:
class LinearClassifier:
    def __init__(self) -> None:
        self.acc1 = 0
        self.acc2 = 0
    '''
    2. Please implement the Linear Classifier from scratch with the update rule in the slide.
    It means you cannot adopt any existing package like sklearn in this assignment. 
    '''
    def LinearClassifier(self, X, Y):
        # 收斂:全部分類正確或是while迴圈次數大於1000次
        convergenceFlag = False
        convergenceNumbers = 0
        iterations = 0
        # learning rate
        lr = 0.001
        # array of number for shuffling
        order = np.arange(0,len(X),1)

        random.seed(1)
        W = np.random.rand(X.shape[1])
        b = np.random.random()

        start_time = time.time()
        while not convergenceFlag:
            iterations += 1
            
            random.shuffle(order)
            for i in order:
                # F = X[i,:].reshape((len(X[i,:])))
                F = X[i,:]
                label = Y[i]
                # check if it's correct based on the current model
                prediction = 1 if (np.dot(W, F) + b) > 0 else -1

                if label * prediction < 0:
                    # update all the weights
                    for j in range(len(W)):
                        W[j] = W[j] + lr * (F[j] * label)
                    b = b + label
                else:
                    convergenceNumbers += 1

            if iterations > 1000 or convergenceNumbers == len(Y):
                convergenceFlag = True

        print('--- {} seconds ---'.format(time.time() - start_time))
        print('--- ||w||^2: {} ---'.format(np.linalg.norm(W)**2))

        # calculate the correction number (accuracy)
        s = 0
        for x, y in zip(X, Y):
            pred = 1 if (np.dot(W, x) + b) > 0 else -1
            if y == pred:
                s += 1
        self.acc1 = s / len(Y)
        print('--- accuracy: {} ---'.format(self.acc1))
    '''
    3. When the "J=WX+b" could be represented as the matrix form for the linear classifier,
    please find the solution by solving this equation using least-squared manner.
    Also, please implement it and make a comparison between this method and the previous one implemented in 2.
    '''
    def LinearClassifierMatrixForm(self, X, Y):
        Y = Y.reshape((len(Y), 1))
        W = np.ones((X.shape[1], 1))
        b = np.random.random()

        start_time = time.time()
        # check if it's correct based on the current model
        beta_hat = np.vstack((b, W))
        designMatrix = np.hstack((np.ones((len(X), 1)), X))
        
        # update all the weights by least squared method
        beta_hat = np.linalg.inv(designMatrix.T @ designMatrix) @ designMatrix.T @ Y

        print('--- {} seconds ---'.format(time.time() - start_time))
        print('--- ||w||^2: {} ---'.format(np.linalg.norm(beta_hat[1:])**2))

        # calculate the correction number (accuracy)
        s = 0
        preds = [1 if item > 0 else -1 for item in (np.dot(designMatrix, beta_hat))]
        for y, pred in zip(Y, preds):
            if y == pred:
                s += 1
        self.acc2 = s / len(Y)
        print('--- accuracy: {} ---'.format(self.acc2))

### Voted Perceptron

In [4]:
class VotedPerceptron:
    def __init__(self, n_iter) -> None:
        self.n_iter = n_iter
        self.W = [] # store the weights
        self.C = [] # store the number of examples that set of weights got correct
        self.B = []

    def fit(self, x, y):
        w = [np.ones_like(x[0])] # store the weights
        b = [0]
        c = [0] # store the number of examples that set of weights got correct
        k = 0
        # learning rate
        lr = 0.001

        # train n_iter epochs
        for epoch in range(self.n_iter):
            # check if each x_i is classified right
            for i in range(len(x)):
                pred = 1 if np.dot(w[k], x[i]) + b[k] > 0 else -1 # checks the sign of w*k
                if pred == y[i]: # checks if the prediction matches the real Y
                    c[k] += 1 # increments c
                else:
                    w.append(np.add(w[k], lr * y[i] * x[i]))
                    b.append(np.add(b[k], y[i]))
                    c.append(0)
                    k += 1

        self.W = w
        self.B = b
        self.C = c
        self.k = k

    def predict(self, X):
        # calculate the prediction from ALL saved weights
        # multiply each prediction by the number it got correct (i.e a weighted vote) and take the sum over all predictions
        # said another way: pick whichever prediction has the most votes
        preds = []
        for x in X:
            s = 0
            for w, b, c in zip(self.W, self.B, self.C):
                s = s + c * (np.dot(w, x) + b)
            preds.append(np.sign(1 if s >= 0 else -1))
        return preds


### Support Vector Machine

In [17]:
class SVM:
    def __init__(self) -> None:
        self.acc1 = 0
        self.acc2 = 0
    '''
    5. With minimizing the ||w||^2, it should drive the marginal to be maximized as well.
    Please implement the linear classifier with the minimum ||w||^2 property and verify
    whether the margin of this version is larger than that of the conventional linear classifier or not. 
    '''
    def SVM(self, X, Y):
        # objective function: ||w||^2
        # gradient descent
        # constraint: y(wx+b)>=1

        Y = Y.reshape((len(Y), 1))
        w = np.ones_like(X[0])
        b = np.random.random()
        # learning rate
        lr = 0.01

        iter = 0
        start_time = time.time()
        while iter < 100:
            for i in range(len(X)):
                """
                Cost Function:
                    l(w) = sum(max(0, 1 - y(wX + b))) + (lambda/2)(||w||)^2
                    First Term is Hinge Loss, Second is Regularization term
                Gradient:
                    delta_w = dl/dw = (1/n) * ( if y(wX+b) < 1: -yX + lambda*w else: lambda*w )
                Gradient Descent
                    w = w - (learning_rate * delta_w)
		        """
                pred = 1 if (np.dot(w, X[i]) + b) > 0 else -1
                loss = np.linalg.norm(w, ord=2)**2
                delta = 1e-4
                if pred * Y[i] < 1:
                    for j in range(len(w)):
                        w[j] -= lr * (-Y[i] * X[i,j]) * delta
                    b = b + Y[i]
            
            iter += 1

        print('--- {} seconds ---'.format(time.time() - start_time))
        print('--- ||w||^2: {} ---'.format(np.linalg.norm(w)**2))

        # calculate the correction number (accuracy)
        s = 0
        for x, y in zip(X, Y):
            pred = 1 if (np.dot(w, x) + b) > 0 else -1
            if y == pred:
                s += 1
        self.acc1 = s / len(Y)
        print('--- accuracy: {} ---'.format(self.acc1))

    '''
    6. Based on 5, please add the slack variable term in the linear classifier and find the most effective weighting value C.
    '''
    def SVM2(self, X, Y):
        # objective function: ||w||^2
        # gradient descent
        # constraint: y(wx+b)>=1
        # with slack variables

        Y = Y.reshape((len(Y), 1))
        w = np.ones_like(X[0])
        b = np.random.random()
        # learning rate
        lr = 0.01
        # constant of slack variables
        C = 0.1
        # slack variables
        slackVariables = np.zeros(len(X))

        iter = 0
        start_time = time.time()
        while iter < 100:
            for i in range(len(X)):
                pred = 1 if (np.dot(w, X[i]) + b) >= 0 else -1

                # compute slack variables
                for k in range(len(X)):
                    pred = 1 if (np.dot(w, X[k]) + b) >= 0 else -1
                    slackVariables[i] == max(0, 1 - Y[i] * pred)

                loss = np.linalg.norm(w, ord=2)**2 + C * np.sum(slackVariables)
                delta = 1e-4
                if pred * Y[i] < 1:
                    # first_order_loss = np.array([2 * w[j] + C * np.dot(X[:,j], Y) for j in range(len(w))]).reshape(len(X[0])) # gradient of loss
                    # w = w - lr * first_order_loss
                    for j in range(len(w)):
                        w[j] -= lr * (-Y[i] * X[i,j]) * delta
                    b = b + Y[i]

            iter += 1

        print('--- {} seconds ---'.format(time.time() - start_time))
        print('--- ||w||^2: {} ---'.format(np.linalg.norm(w)**2))

        # calculate the correction number (accuracy)
        s = 0
        for x, y in zip(X, Y):
            pred = 1 if (np.dot(w, x) + b) >= 0 else -1
            if y == pred:
                s += 1
        self.acc2 = s / len(Y)
        print('--- accuracy: {} ---'.format(self.acc2))

## "data.csv"

### Data Extraction

In [6]:
file_path = './data.csv'
X, Y = DataPreprocess().data_prepare_1(file_path)

### Result of Linear Classifier

In [12]:
classifier = LinearClassifier()

Linear Classifier

In [8]:
classifier.LinearClassifier(X, Y)

--- 1.9354827404022217 seconds ---
--- ||w||^2: 4364.791920036971 ---
--- accuracy: 0.9191564147627417 ---


Linear Classifier with Matrix Form

In [13]:
classifier.LinearClassifierMatrixForm(X, Y)

--- 0.001001596450805664 seconds ---
--- ||w||^2: 1895.4506394416992 ---
--- accuracy: 0.9648506151142355 ---


### Result of Voted Perceptron

In [15]:
from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report

v_perc = VotedPerceptron(n_iter=100)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
v_perc.fit(X_train, Y_train)
Ypred = v_perc.predict(X_test)
# Y_tolist = Y.tolist()
print('Accuracy score: ', accuracy_score(Y_test, Ypred))
# print('Metrics Report')
# print(classification_report(Y_tolist, Ypred))

Accuracy score:  0.958041958041958


### Result of SVM

In [18]:
svm = SVM()

without Slack Variables

In [15]:
svm.SVM(X, Y)

--- 1.7730846405029297 seconds ---
--- ||w||^2: 27.27858511144094 ---
--- accuracy: 0.8787346221441125 ---


with Slack Variables

In [19]:
svm.SVM2(X, Y)

--- 392.202011346817 seconds ---
--- ||w||^2: 80.37248564241975 ---
--- accuracy: 0.38488576449912126 ---


### Compare to Any Existing SVM Package

sklearn.svm

In [29]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

clf = sklearn.svm.SVC()
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.9440559440559441

---

## "crx.csv"

### Data Extraction

In [20]:
file_path = './crx.csv'
X, Y = DataPreprocess().data_prepare_2(file_path)

### Result of Linear Classifier

In [21]:
classifier = LinearClassifier()

Linear Classifier

In [22]:
classifier.LinearClassifier(X, Y)

--- 2.8493974208831787 seconds ---
--- ||w||^2: 17035.660831605295 ---
--- accuracy: 0.6666666666666666 ---


Linear Classifier with Matrix Form

In [23]:
classifier.LinearClassifierMatrixForm(X, Y)

--- 0.0009999275207519531 seconds ---
--- ||w||^2: 3.4659748636273007e+33 ---
--- accuracy: 0.5260869565217391 ---


### Result of Voted Perceptron

In [23]:
from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report

v_perc = VotedPerceptron(n_iter=100)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
v_perc.fit(X_train, Y_train)
Ypred = v_perc.predict(X_test)
# Y_tolist = Y.tolist()
print('Accuracy score: ', accuracy_score(Y_test, Ypred))
# print('Metrics Report')
# print(classification_report(Y_tolist, Ypred))

Accuracy score:  0.6936416184971098


### Result of SVM

In [24]:
svm = SVM()

without Slack Variables

In [25]:
svm.SVM(X, Y)

--- 1.9575891494750977 seconds ---
--- ||w||^2: 12.722853592062004 ---
--- accuracy: 0.663768115942029 ---


with Slack Variables

In [26]:
svm.SVM2(X, Y)

--- 356.1844391822815 seconds ---
--- ||w||^2: 156.49774122955563 ---
--- accuracy: 0.4449275362318841 ---


### Compare to Any Existing SVM Package

sklearn.svm

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

clf = sklearn.svm.SVC()
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

0.7167630057803468